# Lab 1-4: Hello World Pipeline

간단한 덧셈과 곱셈을 수행하는 첫 번째 Kubeflow Pipeline을 작성합니다.

## 파이프라인 구조

```
┌─────────┐     ┌──────────┐     ┌──────────────┐
│   add   │ ──▶ │ multiply │ ──▶ │ print_result │
│ (a + b) │     │  (x * 2) │     │   (출력)     │
└─────────┘     └──────────┘     └──────────────┘
```

## 1. Import 및 설정

In [ ]:
# KFP SDK import
import kfp
from kfp import dsl
from kfp.components import create_component_from_func
from kfp import compiler

print(f"KFP Version: {kfp.__version__}")

## 2. Component 정의

### Component 1: add

In [ ]:
@create_component_from_func
def add(a: int, b: int) -> int:
    """
    두 숫자를 더합니다.
    
    Args:
        a: 첫 번째 숫자
        b: 두 번째 숫자
    
    Returns:
        a + b의 결과
    """
    result = a + b
    print(f"Add Component: {a} + {b} = {result}")
    return result

### Component 2: multiply

In [ ]:
@create_component_from_func
def multiply(x: int, factor: int = 2) -> int:
    """
    숫자에 factor를 곱합니다.
    
    Args:
        x: 입력 숫자
        factor: 곱할 값 (기본값: 2)
    
    Returns:
        x * factor의 결과
    """
    result = x * factor
    print(f"Multiply Component: {x} * {factor} = {result}")
    return result

### Component 3: print_result

In [ ]:
@create_component_from_func
def print_result(value: int):
    """
    최종 결과를 출력합니다.
    
    Args:
        value: 출력할 값
    """
    print("=" * 50)
    print(f"  🎉 Final Result: {value}")
    print("=" * 50)

## 3. Pipeline 정의

In [ ]:
@dsl.pipeline(
    name='Hello World Pipeline',
    description='간단한 덧셈과 곱셈을 수행하는 첫 번째 파이프라인'
)
def hello_pipeline(
    a: int = 3,
    b: int = 5,
    factor: int = 2
):
    """
    Hello World Pipeline
    
    계산: (a + b) * factor
    """
    
    # Step 1: a + b 계산
    add_task = add(a=a, b=b)
    
    # Step 2: (a + b) * factor 계산
    multiply_task = multiply(
        x=add_task.output,
        factor=factor
    )
    
    # Step 3: 결과 출력
    print_result(value=multiply_task.output)

print("✅ Pipeline 정의 완료!")

## 4. Pipeline 컴파일

In [ ]:
# 파이프라인을 YAML 파일로 컴파일
pipeline_file = 'hello_pipeline.yaml'

compiler.Compiler().compile(
    pipeline_func=hello_pipeline,
    package_path=pipeline_file
)

print(f"✅ Pipeline compiled: {pipeline_file}")

In [ ]:
# 생성된 YAML 파일 확인
!head -50 hello_pipeline.yaml

## 5. Pipeline 실행

In [ ]:
# Kubeflow Pipeline Client 생성
client = kfp.Client()

print(f"KFP Host: {client._host}")
print(f"Namespace: {client.get_user_namespace()}")

In [ ]:
# 파이프라인 실행
run = client.create_run_from_pipeline_func(
    hello_pipeline,
    arguments={
        'a': 10,
        'b': 20,
        'factor': 3
    },
    experiment_name='hello-experiment',
    run_name='hello-run-001'
)

print("\n" + "=" * 60)
print("  ✅ Pipeline submitted successfully!")
print("=" * 60)
print(f"\n  Run ID: {run.run_id}")
print(f"  Expected Result: (10 + 20) * 3 = 90")
print("\n💡 Go to Kubeflow Dashboard → Runs to see the results")

## 6. 결과 확인

1. Kubeflow Dashboard에서 **Runs** 메뉴 클릭
2. `hello-run-001` 선택
3. **Graph** 탭에서 DAG 확인
4. 각 컴포넌트 클릭 → **Logs** 탭에서 출력 확인

### 예상 로그

**add 컴포넌트:**
```
Add Component: 10 + 20 = 30
```

**multiply 컴포넌트:**
```
Multiply Component: 30 * 3 = 90
```

**print_result 컴포넌트:**
```
==================================================
  🎉 Final Result: 90
==================================================
```

## 7. (선택) 다른 파라미터로 실행

In [ ]:
# 다른 파라미터로 실행
run2 = client.create_run_from_pipeline_func(
    hello_pipeline,
    arguments={
        'a': 7,
        'b': 3,
        'factor': 5
    },
    experiment_name='hello-experiment',
    run_name='hello-run-002'
)

print(f"Run ID: {run2.run_id}")
print(f"Expected Result: (7 + 3) * 5 = 50")

## ✅ Lab 1-4 완료!

축하합니다! 첫 번째 Kubeflow Pipeline을 성공적으로 작성하고 실행했습니다.

### 학습한 내용
- `@create_component_from_func` 데코레이터로 컴포넌트 정의
- `@dsl.pipeline` 데코레이터로 파이프라인 정의
- `compiler.Compiler().compile()`로 YAML 컴파일
- `kfp.Client()`로 파이프라인 실행

### 다음 단계
- Day 2: 모델 서빙 (FastAPI, MLflow, KServe)